In [20]:
import torch
device = 'cuda' if torch.cuda.is_available() else 

In [13]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW YO


In [14]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [18]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print(encode('hello'))

[61, 58, 65, 65, 68]


In [19]:
encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


In [21]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,  0,
         1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,  1,
         1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50,
        25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38,
        28,  1, 39, 30,  1, 39, 50,  9,  1, 39])


In [23]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [25]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('When input is ', context, ' target is ', target)

When input is  tensor([28])  target is  tensor(39)
When input is  tensor([28, 39])  target is  tensor(42)
When input is  tensor([28, 39, 42])  target is  tensor(39)
When input is  tensor([28, 39, 42, 39])  target is  tensor(44)
When input is  tensor([28, 39, 42, 39, 44])  target is  tensor(32)
When input is  tensor([28, 39, 42, 39, 44, 32])  target is  tensor(49)
When input is  tensor([28, 39, 42, 39, 44, 32, 49])  target is  tensor(1)
When input is  tensor([28, 39, 42, 39, 44, 32, 49,  1])  target is  tensor(25)
